In [1]:
import os
import tempfile
import shutil
import subprocess
from tqdm import tqdm
from PIL import Image
from io import BytesIO
import numpy as np
from rembg import remove
import cv2

In [ ]:
# Function to extract frames from a video using FFmpeg
def extract_frames(video_path: str, output_directory: str, fps: int) -> bool:
    os.makedirs(output_directory, exist_ok=True)
    command = [
        'ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', video_path,
        '-vf', f'fps={fps}', os.path.join(output_directory, 'frame_%04d.png')
    ]
    result = subprocess.run(command)
    return result.returncode == 0

In [ ]:
# Function to remove the background from an image
def remove_background(image_path: str) -> Image.Image:
    with open(image_path, 'rb') as img_file:
        input_image = img_file.read()
    output_image = remove(input_image)
    return Image.open(BytesIO(output_image))


In [ ]:
# Function to overlay foreground image on background image
def overlay_images(fg_image: Image.Image, bg_image: Image.Image) -> Image.Image:
    fg_image = fg_image.convert("RGBA")
    bg_image = bg_image.convert("RGBA")
    # Resize the foreground to match the background
    fg_image = fg_image.resize(bg_image.size, Image.LANCZOS)
    combined = Image.alpha_composite(bg_image, fg_image)
    return combined

In [ ]:
# Create temporary directories
temp_dirs = [tempfile.mkdtemp() for _ in range(4)]
fg_frames_dir, bg_frames_dir, masked_dir, processed_frames_dir = temp_dirs

In [ ]:
# Paths to input videos
fg_video = "E:\\My projects\\Dynamic_Rembg\\input videos\\foreground_video.mp4"  # Placeholder, update with actual name
bg_video = "E:\\My projects\Dynamic_Rembg\\input videos\\background_video.mp4"

In [ ]:
# Extract frames
fps = 24  # Placeholder, update with actual fps if needed
extract_frames(fg_video, fg_frames_dir, fps)
extract_frames(bg_video, bg_frames_dir, fps)

In [ ]:
# Process foreground frames to remove background
fg_frame_files = sorted(os.listdir(fg_frames_dir))
for frame_file in tqdm(fg_frame_files, desc="Processing foreground frames"):
    fg_frame_path = os.path.join(fg_frames_dir, frame_file)
    masked_image = remove_background(fg_frame_path)
    masked_image.save(os.path.join(masked_dir, frame_file))


In [ ]:
# Combine foreground and background frames
bg_frame_files = sorted(os.listdir(bg_frames_dir))
for fg_frame_file, bg_frame_file in tqdm(zip(fg_frame_files, bg_frame_files), desc="Combining frames", total=len(fg_frame_files)):
    fg_frame_path = os.path.join(masked_dir, fg_frame_file)
    bg_frame_path = os.path.join(bg_frames_dir, bg_frame_file)
    fg_image = Image.open(fg_frame_path)
    bg_image = Image.open(bg_frame_path)
    combined_image = overlay_images(fg_image, bg_image)
    combined_image.save(os.path.join(processed_frames_dir, fg_frame_file))

In [ ]:
# Merge processed frames into final video
output_video = 'output_video.mp4'  # Placeholder, update with desired output path
command = [
    'ffmpeg', '-hide_banner', '-loglevel', 'error', '-framerate', str(fps),
    '-i', os.path.join(processed_frames_dir, 'frame_%04d.png'),
    '-c:v', 'libx264', '-pix_fmt', 'yuv420p', output_video
]
subprocess.run(command)

In [ ]:
# Cleanup temporary directories
for temp_dir in temp_dirs:
    shutil.rmtree(temp_dir)